<a href="https://colab.research.google.com/github/Cnava8554/Big-Data-ETL/blob/main/part_one_pets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [920 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-16 01:13:14--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2023-03-16 01:13:15 (10.4 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:


# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/big-data-etl-3.13.2023/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True)

df = df.drop_duplicates()

df = df.dropna()

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   11232277|R1001KC9GI6ARJ|B0080YGXPE|     551772997|AETERTEK AT-918C ...|    Pet Products|          4|            1|          1|   N|                Y|great without gloves|works good keeps ...| 2015-04-16|
|         US|   18802434|R100B4FHILXAWG|B007TRREJC|      56313707|   Happy Pet Harness|    Pet Products|          2|    

In [5]:
# Get the number of rows in the DataFrame.
df.count()

2643241

# Transform the Data

## Create the "review_id_table".

In [6]:
from pyspark.sql.functions import to_date, col
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

review_id_df = df.select("review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date"))

review_id_df = review_id_df.withColumn("customer_id", col("customer_id").cast("integer")).withColumn("product_parent", col("product_parent").cast("integer"))

review_id_df.show()



+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1001KC9GI6ARJ|   11232277|B0080YGXPE|     551772997| 2015-04-16|
|R100B4FHILXAWG|   18802434|B007TRREJC|      56313707| 2014-10-19|
|R1017Z5WWPSSLA|    5584111|B0006342AU|     735846008| 2015-02-02|
|R1019WUJBM9K3X|   32939702|B005JW5VRG|     534045437| 2015-01-20|
|R101IZCRPIN8SX|   50738127|B0029PY7SK|     383930100| 2014-12-03|
|R101KD4S1R47FR|   49683154|B002CM941K|     938528747| 2014-01-28|
|R102EATCBUUDL6|   42553668|B0018KVHWQ|     206118958| 2014-08-08|
|R102OI838NN3Z7|   11056168|B000YDUAMQ|     180607962| 2014-01-18|
|R102VOSABZP429|   45566045|B00TJ3N916|     521682976| 2015-07-31|
|R103GQY0RP7GOH|   45241970|B00CX6LJ22|     595786155| 2014-09-21|
|R103N9CYVCM7YY|   11637795|B00IBWZYVE|     882938526| 2014-06-17|
|R103RPAXH53V7L|   20885662|B0070S62U2|     690582944| 2015-06

## Create the "products" Table

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
from pyspark.sql.functions import to_date, col, substring
products_df = df.select("product_id", substring(col("product_title"), 1,300).alias("product_title"))

products_df = products_df.drop_duplicates()

products_df.show()



+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00134HSYS|Special Edition P...|
|B00BS6NPBG|High Tech Pet 6-P...|
|B000F930FS|PetSafe ScoopFree...|
|B000FJ9QTW|Insight ActiviToy...|
|B000ALY0OQ|SmartCat Bootsie'...|
|B00Q8ETIZ0|Dogloveit Rubber ...|
|B002VU2BA4|Kragonfly Interch...|
|B00QA3K3QM|Attmu Retractable...|
|B005DGHUC2|Zoo Med Laborator...|
|B000XY7C7C|All Four Paws, Th...|
|B004UUE26O|Animal Planet PET...|
|B008APML2C|Chuckit Medium Ul...|
|B00ZJN7T8E|1 Half Portion Ja...|
|B00NRZC8LY|Hide-A-Toy Hallow...|
|B001P3NU4E|Virbac C.E.T. Enz...|
|B003TEQ2U6|Jolly Pets Jolly ...|
|B00DJSNF0M|Multipet Lenny th...|
|B00FXVFEQG|Bags on Board Dur...|
|B005ORDWYA|Just One Bite II ...|
|B003E77OG4|  Petmate Sky Kennel|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [8]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count").withColumn("customer_id", col("customer_id").cast("integer"))

customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   20214846|            15|
|   31120744|             4|
|   20223672|             2|
|   39043742|             1|
|   15231334|             9|
|   44700871|             2|
|   26617857|             7|
|   51750322|             1|
|   42655488|             1|
|   48278478|             1|
|   11417844|             1|
|   34795930|             4|
|   12369110|             8|
|   20437114|             1|
|   25490618|             3|
|   31754174|             6|
|   20460479|             7|
|   22219042|            30|
|    4909697|             2|
|   15741843|             9|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [9]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine") \
            .withColumn("star_rating", col("star_rating").cast("integer")) \
            .withColumn("helpful_votes", col("helpful_votes").cast("integer")) \
            .withColumn("total_votes", col("total_votes").cast("integer"))
            
vine_df.show()



+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1001KC9GI6ARJ|          4|            1|          1|   N|
|R100B4FHILXAWG|          2|            0|          0|   N|
|R1017Z5WWPSSLA|          5|            0|          0|   N|
|R1019WUJBM9K3X|          5|            0|          0|   N|
|R101IZCRPIN8SX|          4|            0|          0|   N|
|R101KD4S1R47FR|          5|            0|          0|   N|
|R102EATCBUUDL6|          5|            0|          1|   N|
|R102OI838NN3Z7|          5|            0|          0|   N|
|R102VOSABZP429|          5|            2|          2|   N|
|R103GQY0RP7GOH|          5|            0|          0|   N|
|R103N9CYVCM7YY|          3|            4|          6|   N|
|R103RPAXH53V7L|          5|            0|          0|   N|
|R103WGJI3K7XR0|          5|            6|          8|   N|
|R103WY0OHHL7SE|          5|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS


In [ ]:
# Write products_df to table in RDS


In [ ]:
# Write customers_df to table in RDS


In [ ]:
# Write vine_df to table in RDS
